## ESPN NBA 球員個人資料、賽事紀錄與成績排序
### 本範例欲爬的目標網站：[ESPN NBA Player Stats](http://www.espn.com/nba/statistics/player/_/stat/assists/sort/avgAssists/)

##  ※僅抓取網站主頁球員資料

In [ ]:
# 可以去掉 python 輸出時，因為軟體版本所引起的警告的警告。
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import requests

from bs4 import BeautifulSoup

import time

In [ ]:
# 以下兩個 url 位址，請任意二選一

#url ='http://www.espn.com/nba/statistics/player/_/stat/assists/sort/avgAssists/'   
url = 'https://www.espn.com/nba/stats/player'

headers= {'User-Agent': 'Mozilla/5.0'}    # 此行可以省略

In [ ]:
response = requests.get(url, headers = headers)

In [ ]:
response.status_code

In [ ]:
response.text

In [ ]:
import pandas as pd

tables = pd.read_html(response.text)

print('找出 %02d 個表格' % len(tables))

#
# 看一下第一個表格內容
#
print('\n\n第一個表格：\n')

df1 = tables[0]

df1.index += 1

print(df1)

In [ ]:
#
# 看一下第二個表格內容
#
print('\n\n第二個表格：\n')

df2 = tables[1]

df2.index += 1

print(df2)

In [ ]:
#
# 將 df1 與 df2 兩個 dataframe(表格)左右水平方向合併成一個新的 dataframe(表格)
# 這裡的 sort = False 的目的是要關閉警告訊息
#
df3 = pd.concat([df1, df2], sort = False, axis = 1)    

#
# 將 df3 的索引值設為 RK 欄位內的數值資料
#
df3.set_index('RK', inplace = True)   

print(df3, '\n')

In [ ]:
# 設定在 Jypyter Notebook 中可以看到原表格內的最大欄位數目
pd.set_option('display.max.columns', 23)   

# 設定在 Jypyter Notebook 中可以看到原表格內的最大列的數目 
pd.set_option('display.max.rows', 50)       

# 如果要讓上面的 set_option 效果可以呈現的話，一定要如此寫；如使用 print(df3)則無效
df3    

# =============================================================

### <p style="line-height:1.6em;" align="justify;"><font color=blue>※ 以下部分為正式抓取網站上所有頁面上球員的資料動作（2021-22 目前共有 581 位球員，每頁有 50 位球員資料，共計 12 頁！ 請注意，這部分資訊會隨時間動態改變。）</font></p>

#### 使用 Selenium Web Scraping 套件<br><br>1. [Selenium with Python 中文手冊](https://selenium-python-zh.readthedocs.io/en/latest/index.html)<br><br>2. [Selenium Chrome Webdriver 下載](https://chromedriver.chromium.org/downloads)<br><br>3. 相關參考 [[1]](https://jzchangmark.wordpress.com/2015/02/26/selenium-%E4%BD%BF%E7%94%A8-chrome-%E7%80%8F%E8%A6%BD%E5%99%A8-webdriver/) [[2]](https://www.youtube.com/watch?v=soPZ4rqlXWI) [[3]](https://medium.com/@bob800530/selenium-1-%E9%96%8B%E5%95%9Fchrome%E7%80%8F%E8%A6%BD%E5%99%A8-21448980dff9) [[4]](https://stackoverflow.max-everyday.com/2018/03/selenium-chrome-webdriver/) [[5]](https://officeguide.cc/windows-python-selenium-automation-scripts-tutorial-examples/)

In [ ]:
# 加裝 selenium
!pip install selenium

#### <p style="line-height:1.6em;" align="justify;"><font color='FF0000'>切記！以下這一段您使用的版本務必要與現在 Chrome 瀏覽器使用的版本一致，Chrome 的 webdriver 可至 https://chromedriver.chromium.org/downloads 中下載至目前的工作目錄路徑下。</font></p>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service


# 以下兩個 url 位址，請任意二選一

#url ='http://www.espn.com/nba/statistics/player/_/stat/assists/sort/avgAssists/'   
url = 'https://www.espn.com/nba/stats/player'

# 啟動 Chrome Browser， 'chromedriver.exe'與本 notebook 程式在同一路徑之下

#browser = webdriver.Chrome()  

service = Service(executable_path = './chromedriver.exe')

browser = webdriver.Chrome(service = service)


# 連接目的地網頁伺服器
browser.get(url)   

# 設定等待頁面加載完成的隱式等待時間限制
Waiting_Timeout = 10   

while True:

    try:
        
        # 等待頁面加載完成
        button = WebDriverWait(browser, Waiting_Timeout).until(EC.presence_of_element_located((By.LINK_TEXT, 'Show More')))
        
        # 由 selenium 代替人類按網頁上的 "Show More" 按鈕
        button.click()
        
        # 共有 12 頁，第一頁不算，應該會執行按鈕 11 次
        print("SHOW/LOAD MORE RESULTS button clicked")  
    
    except TimeoutException:
        
        # 最後 10 秒後會遇上 selenium.common.exceptions.TimeoutException
        print("No more SHOW/LOAD MORE RESULTS button to be clicked")
        
        break

### 請記得此時不可以關閉 ESPN NBA 網頁的瀏覽器（在本例子中所使用的是 Chrome 瀏覽器）

In [ ]:
import pandas as pd

tables = pd.read_html(browser.page_source)

print('\n找出 %02d 個表格' % len(tables), '\n\n')

df1 = tables[0]  # 第一個表格

df2 = tables[1]  # 第二個表格

#
# 將 df1 與 df2 兩個 dataframe(表格)左右水平方向合併成一個新的 dataframe(表格)
# 這裡的 sort = False 的目的是要關閉警告訊息
#
df3 = pd.concat([df1, df2], sort = False, axis = 1)  # sort = False 關閉警告訊息

df3.set_index('RK', inplace = True) 

pd.set_option('display.max.columns', 24)

pd.set_option('display.max.rows', None)

df3

In [ ]:
# 關閉瀏覽器與其 driver
browser.quit()    

In [ ]:
# browser.close() vs browser.quit()
# 結論：用 quit() 方法結束連線較為安全、完整與乾淨
#

print()

help(browser.close)

print('\n============================================================\n')

help(browser.quit)